# Notebook para el re-entrenamiento de RoBERTa

Este notebook está diseñado para entrenar un modelo de clasificación de texto utilizando la biblioteca `transformers` de Hugging Face, específicamente con el modelo `RobertaForSequenceClassification` para la tarea de clasificación de emociones en texto. El conjunto de datos utilizado es GoEmotions, un conjunto de datos de emociones en texto.

## Sección  1: Importación de librerías y configuración inicial

- Se importan las librerías necesarias: `transformers` para el modelo y el tokenizador, `datasets` para cargar y manipular el conjunto de datos, `torch` para el procesamiento y el entrenamiento del modelo, y `numpy` para la manipulación de arrays.
- Se define el nombre del modelo `roberta-base` y la carpeta donde se guardará el modelo entrenado.

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, pipeline, AutoConfig
from datasets import load_dataset
import torch
import numpy as np

In [ ]:
model_name = 'roberta-base'
model_folder = 'roberta-goemotions/'

## Sección  2: Definición de emociones de interés

- Se establece un conjunto de emociones que se desean clasificar.
- Se crea un conjunto de emociones no útiles que se excluyen del análisis.
- Se calcula el conjunto de emociones de interés restando las no útiles del conjunto total de emociones.


In [ ]:
emotions = {'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 
            'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'}
non_useful_emotions = {'disgust','disappointment','grief','disapproval', 'amusement', 'excitement', 'optimism','caring', 'pride', 'admiration', 'relief', 'approval', 'realization', 'surprise', 'curiosity', 'remorse', 'embarrassment', 'neutral'}
emotions_of_interest = emotions - non_useful_emotions
emotions_of_interest

## Sección  3: Carga y preparación del conjunto de datos

- Se carga el conjunto de datos GoEmotions en su formato bruto y se divide en conjuntos de entrenamiento y prueba.
- Se define una función para filtrar columnas que no son útiles y otra para convertir las emociones en etiquetas numéricas.
- Se filtran las columnas no útiles y se aplica la función de etiquetado.
- Se filtra el conjunto de datos para incluir solo las filas que contienen al menos una emoción de interés.

In [ ]:
# Load the GoEmotions dataset
dataset = load_dataset("go_emotions", "raw", split='train')
#dataset = dataset.train_test_split(test_size=0.2)
dataset = dataset.train_test_split(train_size=100, test_size=100)
dataset

In [ ]:
def filter_columns(column_name):
    return column_name not in useful_columns

def labelize(dataset):
    return {"labels": [dataset[column] for column in emotions]}

def filter_row(dataset):
    return 1 in dataset['labels']

useful_columns = emotions.copy()
useful_columns.add('text')
non_useful_columns = dataset['train'].column_names
non_useful_columns = filter(filter_columns, non_useful_columns)
non_useful_columns = list(non_useful_columns)
dataset = dataset.remove_columns(non_useful_columns)
dataset = dataset.map(labelize)
dataset = dataset.filter(filter_row)
dataset

In [ ]:
dataset['test'][0]

## Sección  4: Tokenización y preparación del conjunto de datos

- Se inicializa el tokenizador `RobertaTokenizer` con el modelo `roberta-base`.
- Se define una función para tokenizar el texto del conjunto de datos.
- Se aplica la tokenización al conjunto de datos y se elimina el texto original.

In [ ]:
# Tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained(model_name)

In [ ]:
def tokenize(dataset):
  return tokenizer(dataset['text'], truncation=True)

dataset_columns = dataset["train"].column_names
dataset_columns.remove('labels')
encoded_dataset = dataset.map(tokenize, batched=True, remove_columns=dataset_columns)
encoded_dataset

In [ ]:
encoded_dataset['test'][0]

## Sección  5: Reformateo de las etiquetas

- Se define una función para convertir las etiquetas de emociones en un formato numérico.
- Se aplica esta función al conjunto de datos y se renombra la columna de etiquetas.

In [ ]:
def reformat(dataset):
    array = dataset["labels"]
    np_array = np.asarray(array)
    torch_array = torch.from_numpy(np_array)
    torch_array = torch_array.to(torch.float)
    return {"float_labels": torch_array}

# cast label IDs to floats
encoded_dataset.set_format("torch")
encoded_dataset = encoded_dataset.map(reformat, remove_columns=["labels"])
encoded_dataset = encoded_dataset.rename_column("float_labels", "labels")
encoded_dataset

In [ ]:
encoded_dataset['test'][0]

## Sección  6: Configuración del modelo y argumentos de entrenamiento

- Se crea un mapeo de ID a etiqueta para las emociones.
- Se configura el modelo `RobertaForSequenceClassification` con la configuración personalizada y el mapeo de ID a etiqueta.
- Se establecen los argumentos de entrenamiento, incluyendo el número de épocas, el tamaño del lote, la tasa de aprendizaje, y la estrategia de evaluación.


In [ ]:
# Create an id2label mapping
id2label = {i: label for i, label in enumerate(emotions)}

config = AutoConfig.from_pretrained(model_name)
config.update({"id2label": id2label})
config.update({"problem_type": "multi_label_classification"})
config

In [ ]:
# load model
model = RobertaForSequenceClassification.from_pretrained(model_name, config=config)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir=model_folder,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    learning_rate=2e-5,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=100,
)
training_args = TrainingArguments(model_folder, num_train_epochs=1)

## Sección  7: Entrenamiento del modelo

- Se inicializa el entrenador `Trainer` con el modelo, los argumentos de entrenamiento, y los conjuntos de datos de entrenamiento y prueba.
- Se entrena el modelo y se evalúa su rendimiento.
- Se guarda el modelo entrenado y el tokenizador en la carpeta especificada.


In [ ]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    tokenizer=tokenizer
)

In [ ]:
# Train the model
trainer.evaluate()

In [ ]:
trainer.train()

In [ ]:
# Save the model
model.save_pretrained(model_folder)
tokenizer.save_pretrained(model_folder)

In [ ]:
## Sección  8: Análisis de sentimientos

- Se utiliza un módulo `LyricSentiment` para analizar el sentimiento de un texto lírico.
- Se compara el resultado del análisis sin procesar con el resultado después de limpiar las emociones.
- Se calcula la diferencia entre las emociones detectadas y las emociones limpiadas.

Este notebook es un ejemplo práctico de cómo utilizar la biblioteca `transformers` para entrenar un modelo de clasificación de texto en la tarea de clasificación de emociones, y cómo aplicar este modelo para analizar el sentimiento de un texto.

In [ ]:
from utils import  LyricSentiment
LyricSentiment.set_model(model_folder)
raw_sentiments = LyricSentiment.__analyze("""
It's been seven hours and fifteen days
Since u took your love away
I go out every night and sleep all day
Since you took your love away
Since you been gone I can do whatever I want
I can see whomever I choose
I can eat my dinner in a fancy restaurant
But nothing
I said nothing can take away these blues
'Cause nothing compares
Nothing compares to you
It's been so lonely without you here
Like a bird without a song
Nothing can stop these lonely tears from falling
Tell me baby where did I go wrong
I could put my arms around every boy I see
But they'd only remind me of you
I went to the doctor and guess what he told me
Guess what he told me
He said girl you better have fun
No matter what you do
But he's a fool
'Cause nothing compares
Nothing compares to you
All the flowers that you planted, mama
In the back yard
All died when you went away
I know that living with you baby was sometimes hard
But I'm willing to give it another try
Nothing compares
Nothing compares to you
Nothing compares
Nothing compares to you
Nothing compares
Nothing compares to you
""")
raw_sentiments

In [ ]:
cleared_emotions = LyricSentiment.sentiments("""
It's been seven hours and fifteen days
Since u took your love away
I go out every night and sleep all day
Since you took your love away
Since you been gone I can do whatever I want
I can see whomever I choose
I can eat my dinner in a fancy restaurant
But nothing
I said nothing can take away these blues
'Cause nothing compares
Nothing compares to you
It's been so lonely without you here
Like a bird without a song
Nothing can stop these lonely tears from falling
Tell me baby where did I go wrong
I could put my arms around every boy I see
But they'd only remind me of you
I went to the doctor and guess what he told me
Guess what he told me
He said girl you better have fun
No matter what you do
But he's a fool
'Cause nothing compares
Nothing compares to you
All the flowers that you planted, mama
In the back yard
All died when you went away
I know that living with you baby was sometimes hard
But I'm willing to give it another try
Nothing compares
Nothing compares to you
Nothing compares
Nothing compares to you
Nothing compares
Nothing compares to you
""")
cleared_emotions

In [ ]:
difference = {key: raw_sentiments[key] - cleared_emotions[key] for key in raw_sentiments if key in cleared_emotions}
difference = dict(sorted(difference.items(), key=lambda dict: dict[1]))
difference